<a href="https://colab.research.google.com/github/Susana471978/Lab-LLM/blob/main/Pruebacongelada(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Desactiva los mensajes de advertencia

In [ ]:
pip install "accelerate>=0.26.0"

In [ ]:
!pip install datasets


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

# Configurar el dispositivo (GPU si está disponible, de lo contrario CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Cargar el modelo y el tokenizer preentrenados
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)  # Mover el modelo a GPU/CPU

# Cargar el dataset desde un archivo CSV
dataset = load_dataset('csv', data_files='/content/news_pruebalisto.csv')

# Dividir el dataset en entrenamiento y validación
dataset = dataset['train'].train_test_split(test_size=0.2, seed=42)
dataset['validation'] = dataset.pop('test')

# Tokenización de los datos
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

# Preprocesar el dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Definir los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',          # directorio para los resultados
    evaluation_strategy="epoch",     # estrategia de evaluación (al final de cada época)
    learning_rate=5e-5,
    per_device_train_batch_size=8,   # tamaño del batch de entrenamiento
    per_device_eval_batch_size=8,    # tamaño del batch de evaluación
    num_train_epochs=5,              # número de épocas
    weight_decay=0.01,               # regularización L2
    push_to_hub=False,               # no enviar al Hub de Hugging Face
)

# Definir el trainer
trainer = Trainer(
    model=model,                         # modelo a entrenar
    args=training_args,                  # argumentos de entrenamiento
    train_dataset=tokenized_datasets['train'],   # dataset de entrenamiento
    eval_dataset=tokenized_datasets['validation'],  # dataset de validación
    compute_metrics=lambda p: {
        'accuracy': (p.predictions.argmax(axis=-1) == p.label_ids).mean()
    },  # Métricas de evaluación
)

# Entrenar el modelo
trainer.train()

# Evaluación del modelo
results = trainer.evaluate()

print(f"Resultados de la evaluación: {results}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: susana-pierre-online (susana-pierre-online-reboot-academy). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Cargar el modelo y el tokenizador
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-uncased", num_labels=2)

# Función para hacer las predicciones
def predict_news(news_list):
    # Tokenizamos las noticias
    encodings = tokenizer(news_list, truncation=True, padding=True, max_length=128, return_tensors="pt")

    # Realizamos las predicciones
    with torch.no_grad():
        outputs = model(**encodings)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=-1)  # Probabilidades para ambas clases
        predictions = torch.argmax(probabilities, dim=-1)  # Clase predicha

    return predictions, probabilities

# Noticias de ejemplo para probar
news_to_test = [
    "Scientists have discovered a new element on the periodic table.",  # 0
    "Renewable energy sources are projected to supply 50% of the world's power by 2050.",  # 1
    "The Earth is flat, according to recent studies.",  # 0
    "A revolutionary cancer treatment has been approved for use worldwide.",  # 1
    "The Great Wall of China is visible from space, as confirmed by astronauts.",  # 0
    "Researchers found a way to extend human life by 50 years using genetic engineering.",  # 0
    "NASA plans to land humans on Mars by the year 2030.",  # 1
    "Eating chocolate increases your intelligence, according to new research.",  # 0
    "The Amazon rainforest produces 20% of the Earth's oxygen, according to scientists.",  # 1
    "A new type of plastic has been invented that is completely biodegradable within 24 hours.",  # 1
    "Aliens have been spotted in a remote desert area, say eyewitnesses.",  # 0
    "Electric cars are becoming more affordable due to advancements in battery technology.",  # 1
    "A recent study shows that plants can talk to each other using chemical signals.",  # 1
    "The moon landing was faked, according to declassified documents."  # 0
]

# Etiquetas esperadas (0 = fake, 1 = true)
true_labels = [0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0]

# Realizamos las predicciones
predictions, probabilities = predict_news(news_to_test)

# Contadores para predicciones correctas e incorrectas
correct_true = 0
correct_false = 0
incorrect_true = 0
incorrect_false = 0

# Mostrar los resultados junto con las etiquetas verdaderas
for i, news in enumerate(news_to_test):
    # Obtenemos el índice de la clase con mayor probabilidad
    pred_class = predictions[i].item()  # Convertimos la predicción a un valor escalar (0 o 1)
    pred_label = 'True' if pred_class == 1 else 'Fake'  # Si es 1 es 'True', si es 0 es 'Fake'

    true_label = 'True' if true_labels[i] == 1 else 'Fake'  # Las etiquetas verdaderas ya son escalares

    # Comparar las predicciones con las etiquetas verdaderas
    if true_labels[i] == 1 and pred_class == 1:
        correct_true += 1
    elif true_labels[i] == 0 and pred_class == 0:
        correct_false += 1
    elif true_labels[i] == 1 and pred_class == 0:
        incorrect_true += 1
    elif true_labels[i] == 0 and pred_class == 1:
        incorrect_false += 1

    # Mostrar resultados
    print(f"News: {news}")
    print(f"Model Prediction: {pred_label}")
    print(f"True Label: {true_label}")
    print(f"Probability of being Fake: {probabilities[i][0]:.4f}")
    print(f"Probability of being True: {probabilities[i][1]:.4f}")
    print("")

# Calcular las métricas de evaluación
predictions_flat = predictions.numpy()  # Convertimos las predicciones a un array de NumPy
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions_flat, average='binary')
accuracy = accuracy_score(true_labels, predictions_flat)

# Mostrar las métricas
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# Mostrar los resultados finales
print("\nFinal Counts:")
print(f"Correct True: {correct_true}")
print(f"Correct Fake: {correct_false}")
print(f"Incorrect True: {incorrect_true}")
print(f"Incorrect Fake: {incorrect_false}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


News: Scientists have discovered a new element on the periodic table.
Model Prediction: True
True Label: Fake
Probability of being Fake: 0.4592
Probability of being True: 0.5408

News: Renewable energy sources are projected to supply 50% of the world's power by 2050.
Model Prediction: True
True Label: True
Probability of being Fake: 0.4773
Probability of being True: 0.5227

News: The Earth is flat, according to recent studies.
Model Prediction: Fake
True Label: Fake
Probability of being Fake: 0.5080
Probability of being True: 0.4920

News: A revolutionary cancer treatment has been approved for use worldwide.
Model Prediction: True
True Label: True
Probability of being Fake: 0.4836
Probability of being True: 0.5164

News: The Great Wall of China is visible from space, as confirmed by astronauts.
Model Prediction: Fake
True Label: Fake
Probability of being Fake: 0.5333
Probability of being True: 0.4667

News: Researchers found a way to extend human life by 50 years using genetic engineer